In [10]:
import pandas               as     pd
import numpy                as     np

from   jupyterlab_dash      import AppViewer
from   dash                 import Dash
from   dash.dependencies    import State, Input, Output
from   dash_html_components import Div, P, Label, Img, H6
from   dash_core_components import Graph, Dropdown, Checklist, Loading
from   plotly.graph_objects import Figure, Layout, Choropleth

scope = pd.read_csv(f'../data/Clean/scope.csv')
corps = scope.company.unique()
areas = scope.region.unique()

def makeMap(company = 'Apple'):

    d     = scope[scope.company == company].groupby('code').sum()

    pMap  = Choropleth(
                locations = d.index,
                z         = d.employees,
                text      = d.index
            )

    lMap  = Layout(
                title = f'Corporate Footprint for {company}',
                geo   = \
                {
                    'showframe'       : False,
                    'showcoastlines'  : False,
                    'projection_type' : 'equirectangular'
                }
            )

    fMap  = Figure(
                data   = pMap,
                layout = lMap
            )

    gMap  = Graph(
                id     = 'map',
                figure = fMap
            )
    
    return gMap

def makeCon() :

    return Div(
        id        = "upper-left",
        className = "six columns",
        children  =
        [
            P(
                className = "section-title",
                children  = "Choose a Corporation and Region(s) to See Risk Exposures",
            ),
            Div(
                className = "control-row-1",
                children  =
                [
                    Div(
                        id       = "state-select-outer",
                        children =
                        [
                            Label("Corporation"),
                            Dropdown(
                                id      = "state-select",
                                options = [{"label": c, "value": c} for c in corps],
                                value   = corps[0],
                            ),
                        ],
                    )
                ],
            ),
            Div(
                id        = "region-select-outer",
                className = "control-row-2",
                children  =
                [
                    Label("Region(s)"),
                    Div(
                        id       = "region-select-dropdown-outer",
                        children =
                        [
                            Dropdown(
                                id         = "region-select",
                                options    = [{"label": a, "value": a} for a in areas],
                                value      = areas,
                                multi      = True,
                                searchable = True
                            )
                        ]
                    ),
                ],
            ),
        ],
    )

def makeRoot() :

    return Div(
        className = "container scalable",
        children  =
        [
            Div(
                id        = "banner",
                className = "banner",
                children  =
                [
                    H6("Global Risk"),
                    Img(src = app.get_asset_url("globe.gif")),
                ],
            ),
            Div(
                id        = "upper-container",
                className = "row",
                children  =
                [
                    makeCon(),
                    Div(
                        id        = "geo-map-outer",
                        className = "six columns",
                        children  =
                        [
                            P(
                                id       = "map-title",
                                children = "Risk Exposure for Apple",
                            ),
                            Div(
                                id       = "geo-map-loading-outer",
                                children =
                                [
                                    Loading(
                                        id       = "loading",
                                        children = Graph(
                                            id     = "geo-map",
                                            figure =
                                            {
                                                "data"   : [],
                                                "layout" :
                                                {
                                                    'plot_bgcolor'  : '#171b26',
                                                    'paper_bgcolor' : '#171b26',
                                                },
                                            },
                                        ),
                                    )
                                ],
                            ),
                        ],
                    ),
                ],
            ),
            Div(
                id       ="lower-container",
                children =
                [
                    Graph(
                        id     = "procedure-plot",
                        figure = makeTrend(),
                    )
                ],
            ),
        ],
    )
    

def makeTrend():
    
    return Figure()
    
lab = AppViewer()
app = Dash(__name__,
           meta_tags =[
           {
               'name'    : 'viewport',
               'content' : 'width=device-width, initial-scale=1, maximum-scale=1.0, user-scalable=no'
           }])

app.layout = makeRoot()

lab.show(app)